In [1]:
pip install opencv-python mediapipe numpy


Defaulting to user installation because normal site-packages is not writeable
  Using cached protobuf-4.25.3-cp37-abi3-manylinux2014_x86_64.whl.metadata (541 bytes)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 MB 5.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.7/35.7 MB 5.8 MB/s eta 0:00:0000:0100:01
Using cached protobuf-4.25.3-cp37-abi3-manylinux2014_x86_64.whl (294 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.7/133.7 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 6.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.6/79.6 MB 5.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.7/68.7 MB 5.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 5.9 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 4.9 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [1]:
import cv2
import mediapipe as mp
import numpy as np


In [2]:

# Function to calculate Euclidean distance between two 3D points
def calculate_distance(point1, point2):
    return np.sqrt((point1.x - point2.x) ** 2 +
                   (point1.y - point2.y) ** 2 +
                   (point1.z - point2.z) ** 2)



In [3]:
# Initialize MediaPipe Hands
mp_hands = mp.solutions.hands
hands = mp_hands.Hands()
mp_drawing = mp.solutions.drawing_utils




libEGL warning: MESA-LOADER: failed to open iris: /opt/anaconda/lib/python3.11/site-packages/zmq/backend/cython/../../../../.././libstdc++.so.6: version `GLIBCXX_3.4.30' not found (required by /usr/lib/dri/iris_dri.so) (search paths /usr/lib/dri, suffix _dri)

libEGL warning: MESA-LOADER: failed to open zink: /opt/anaconda/lib/python3.11/site-packages/zmq/backend/cython/../../../../.././libstdc++.so.6: version `GLIBCXX_3.4.30' not found (required by /usr/lib/dri/zink_dri.so) (search paths /usr/lib/dri, suffix _dri)

libEGL warning: MESA-LOADER: failed to open swrast: /opt/anaconda/lib/python3.11/site-packages/zmq/backend/cython/../../../../.././libstdc++.so.6: version `GLIBCXX_3.4.30' not found (required by /usr/lib/dri/swrast_dri.so) (search paths /usr/lib/dri, suffix _dri)

libEGL warning: MESA-LOADER: failed to open iris: /opt/anaconda/lib/python3.11/site-packages/zmq/backend/cython/../../../../.././libstdc++.so.6: version `GLIBCXX_3.4.30' not found (required by /usr/lib/dri/iris_dr

In [5]:
# Open webcam
cap = cv2.VideoCapture(0)

try:
    while cap.isOpened():
        # Read frame from webcam
        success, image = cap.read()
        if not success:
            print("Failed to read frame from webcam. Exiting...")
            break

        # Convert BGR image to RGB
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # Process the image with MediaPipe Hands
        results = hands.process(image_rgb)

        finger_count = 0

        # If hand landmarks are detected
        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                # Draw hand landmarks on the image
                mp_drawing.draw_landmarks(image, hand_landmarks, mp_hands.HAND_CONNECTIONS)

                # Calculate distances for finger counting
                thumb_tip = hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_TIP]
                thumb_mcp = hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_MCP]

                index_tip = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP]
                index_mcp = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_MCP]

                middle_tip = hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_TIP]
                middle_mcp = hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_MCP]

                ring_tip = hand_landmarks.landmark[mp_hands.HandLandmark.RING_FINGER_TIP]
                ring_mcp = hand_landmarks.landmark[mp_hands.HandLandmark.RING_FINGER_MCP]

                pinky_tip = hand_landmarks.landmark[mp_hands.HandLandmark.PINKY_TIP]
                pinky_mcp = hand_landmarks.landmark[mp_hands.HandLandmark.PINKY_MCP]

                # Calculate distances for each finger segment
                if calculate_distance(thumb_tip, thumb_mcp) > 0.16:
                    finger_count += 1
                if calculate_distance(index_tip, index_mcp) > 0.18:
                    finger_count += 1
                if calculate_distance(middle_tip, middle_mcp) > 0.18:
                    finger_count += 1
                if calculate_distance(ring_tip, ring_mcp) > 0.18:
                    finger_count += 1
                if calculate_distance(pinky_tip, pinky_mcp) > 0.18:
                    finger_count += 1

        # Display finger count on the image
        cv2.putText(
            image,
            f"Finger Count: {finger_count}",
            (10, 30),
            cv2.FONT_HERSHEY_DUPLEX,
            1,
            (0, 255, 0),
            2,
        )

        # Display the annotated image with finger count
        cv2.imshow("Finger Counting", image)

        # Exit the loop if 'q' key is pressed
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break

finally:
    # Release the webcam and close all OpenCV windows
    cap.release()
    cv2.destroyAllWindows()

QObject::moveToThread: Current thread (0x13307750) is not the object's thread (0x11747000).
Cannot move to target thread (0x13307750)

QObject::moveToThread: Current thread (0x13307750) is not the object's thread (0x11747000).
Cannot move to target thread (0x13307750)

QObject::moveToThread: Current thread (0x13307750) is not the object's thread (0x11747000).
Cannot move to target thread (0x13307750)

QObject::moveToThread: Current thread (0x13307750) is not the object's thread (0x11747000).
Cannot move to target thread (0x13307750)

QObject::moveToThread: Current thread (0x13307750) is not the object's thread (0x11747000).
Cannot move to target thread (0x13307750)

QObject::moveToThread: Current thread (0x13307750) is not the object's thread (0x11747000).
Cannot move to target thread (0x13307750)

QObject::moveToThread: Current thread (0x13307750) is not the object's thread (0x11747000).
Cannot move to target thread (0x13307750)

QObject::moveToThread: Current thread (0x13307750) is n

KeyboardInterrupt: 